In [1]:
from cluster_utils import ClustersShots, ClustersTracks
import os
import numpy as np
import cv2
from matplotlib import pyplot as plt
import time
from cofc_utils import face_element, display_cv_image, extract_bboxes_and_features, get_deep_feature, get_dlib_detector, get_face_bboxes_in_frame, initialize_deep_models, shot_boundary

In [3]:
def get_facetracks_and_links(shot_data, th_feats=1.0, th_overlap=0.85):
    t1 = time.time()
    fno = 0
    nFaces = len(shot_data)
    ls_inds = [0]

    ls_cannotlink = []
    cl_tracks = ClustersTracks(simThresh=th_overlap, featThresh=th_feats)
    
    for i in range(1, nFaces):
        
        fno = shot_data[i].fno
        if(fno != shot_data[i-1].fno):
            #do something
            ls_data = [shot_data[x] for x in ls_inds]
            ls_cl = cl_tracks.cluster_online(ls_data)
            ls_cannotlink.append(ls_cl)    
            ls_inds = [i]
        else:
            ls_inds.append(i)

    ls_tracks = cl_tracks.clusters
    l = len(ls_tracks)
    qMatrix = np.ones((l, l))

    for l in ls_cannotlink:
        for i in range(len(l)):
            for j in range(i+1, len(l)):
                qMatrix[i,j] = 0
                qMatrix[j,i] = 0
    t2 = time.time()
    
    #delete tracks with length < 15
    i=0
    while(i<len(ls_tracks)):
        if(len(ls_tracks[i]) < 15):
            del ls_tracks[i]
            qMatrix = np.delete(qMatrix, i, axis=0)
            qMatrix = np.delete(qMatrix, i, axis=1)
            i-=1
        i+=1
    return ls_tracks, qMatrix
    
def process_shot(clusters_shot, ls_frames, detector, aligner, fnet):
    t1 = time.time()
    shot_data = extract_bboxes_and_features(ls_frames, detector, aligner, fnet) # list of face_element
    t2 = time.time()
    print("Shot has %d faces on which OPENFACE took %.3f secs"%(len(shot_data), t2-t1))
    t1=t2
    face_tracks, qMatrix = get_facetracks_and_links(shot_data)
    clusters_shot.cluster_online(face_tracks, qMatrix) #the functionwa in paperwa
    t2 = time.time()
    print("Processing the shot and clustering took %.2f secs"%(t2-t1))
    return face_tracks, qMatrix
    

In [5]:
path = "./will_smith_interview.mp4"
cap = cv2.VideoCapture(path)
aligner, fnet = initialize_deep_models()
detector = get_dlib_detector()

ret = False

ls_frames = []

for i in range(2):
    ret, frame = cap.read()
    assert(ret==True)
    if(i==0):
        ppframe = frame
    if(i==1):
        pframe = frame
    ls_frames.append(frame)

clusters_shot = ClustersShots(3.0, "./clusters")
kk = 0
ft = []
while(ret == True):
    print(kk)
    kk+=1
    # Capture frame-by-frame
    ret, frame = cap.read()
    if(ret==False):
        # No frame detected hence video is ended.
        print("Processing shot of n frames: " + str(len(ls_frames)))
        ft, qmat = process_shot(clusters_shot, ls_frames, detector, aligner, fnet)
        print(qmat)
        ls_frames = []
    else:
        ls_frames.append(frame)
        sb = shot_boundary(ppframe, pframe, frame)
        # If shot boundary is detected or clip is more than 100 frames (assuming framerate ~20-30fps), process it
        if (sb or len(ls_frames) > 24*10): #more than 10s
            print("Processing shot of n frames: " + str(len(ls_frames)))
            ft, qmat = process_shot(clusters_shot, ls_frames, detector, aligner, fnet)
            print(qmat)
            ls_frames = []
        #update prev-prev frame and prev-frame
        ppframe = pframe
        pframe = frame

print("Completed for the video: "+path)

Loading the dlib and OpenFace models took 1.2865011692 seconds.
Creating directory: ./clusters
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
Processing shot of n frames: 134


KeyboardInterrupt: 